In [2]:
UNIV_COLUMNS = ['대학코드', '국가', '대학명', '파견구분', '협정형태', 'NUM_VISITS', "BERT_SUM", "RNN_SUM", 'href']
ABSTRACT_REVIEWS_COLUMNS = ['대학코드', '제목', '학과', '과정', '년도', 'BERT_SCORE', 'RNN_SCORE', 'href']
SPECIFIC_COLUMNS = ["대학코드", "대학명", "국가", "파견구분", "협정형태", "학과", "제목", "과정", "년도", "college_link", "review_link",  "gen_info","env_info","food_info","study_info","office_info" "facil_info","mhct_info","help_info","etc_info", "BERT_SCORE", "RNN_SCORE"]

In [3]:
# import all modules
import os
from os import fdopen, remove
import glob
from tempfile import mkstemp
import shutil
from shutil import move, copymode

import pandas as pd
import numpy as np

## wrangle abstract review dataset

In [4]:
# define path for sentiment labeled dataset
# abstract: 짧은 후기 제목만 있는 텍스트 데이터셋
# specific: 문단으로 구성된 텍스트 데이터셋

abstract_sentiment = "./data_sentiment/abstract"
specific_sentiment = "./data_sentiment/specific"

In [5]:
# fetch all yonsei exchange review text datasets for each foreign universities
abstract_yonsei_reviews = glob.glob(f"{abstract_sentiment}/*.csv")
abstract_yonsei_reviews.sort()
abstract_yonsei_reviews[:5]

['./data_sentiment/abstract/730     _review_abstract_sentiment.csv',
 './data_sentiment/abstract/AR000004_review_abstract_sentiment.csv',
 './data_sentiment/abstract/AT000001_review_abstract_sentiment.csv',
 './data_sentiment/abstract/AT000002_review_abstract_sentiment.csv',
 './data_sentiment/abstract/AT000003_review_abstract_sentiment.csv']

In [6]:
# get sample dataset
sample_file = abstract_yonsei_reviews[70]

# extract file name
file_name = sample_file.split("/")[-1]
file_name_without_ext = file_name.split(".")[0]
print(file_name_without_ext)

# extract university code
univ_code_sample = file_name_without_ext.split("_")[0]
print(univ_code_sample)

# read sample dataset
df_abstract = pd.read_csv(sample_file, encoding="utf-8")
df_abstract.sample(5)

CN000022_review_abstract_sentiment
CN000022


,Unnamed: 0,No,제목,학과,과정,년도,href,BERT_SCORE,RNN_SCORE
8,8,10,북경대학교 경험보고서,아시아학부,학부,2017,/partner/expReport.asp?id=14387&page=1&bgbn=R,1,0.775683
11,11,7,북경대학교 교환생활,중어중문학과,학부,2016,/partner/expReport.asp?id=13586&page=2&bgbn=R,0,0.602057
15,15,3,북경대학교,정치외교학과,학부,2014,/partner/expReport.asp?id=7298&page=2&bgbn=R,1,0.659739
9,9,9,북경대학교 교환학생,중어중문학과,학부,2017,/partner/expReport.asp?id=14304&page=1&bgbn=R,0,0.664221
4,4,14,북경대학교에서의 한 학기 교환생활,경영학과,학부,2018,/partner/expReport.asp?id=15209&page=1&bgbn=R,1,0.697339


In [7]:
df_abstract_review = df_abstract[["제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
df_abstract_review["대학코드"] = univ_code_sample
_abstract_review = df_abstract[["제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
df_abstract_review["대학코드"] = univ_code_sample
df_abstract_review = df_abstract_review[["대학코드","제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
df_abstract_review.head()

,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,href
0,CN000022,PKU Exchange Program,언더우드국제대학,학부,2019,0,0.515649,/partner/expReport.asp?id=16451&page=1&bgbn=R
1,CN000022,베이징대 교환생활,중어중문학과,학부,2019,0,0.484792,/partner/expReport.asp?id=16194&page=1&bgbn=R
2,CN000022,북경대학교 교환학생,정치외교학과,학부,2019,0,0.664221,/partner/expReport.asp?id=15979&page=1&bgbn=R
3,CN000022,북경대학교 교환생활,중어중문학과,학부,2018,0,0.602057,/partner/expReport.asp?id=15513&page=1&bgbn=R
4,CN000022,북경대학교에서의 한 학기 교환생활,경영학과,학부,2018,1,0.697339,/partner/expReport.asp?id=15209&page=1&bgbn=R


In [8]:
df_abstract_review.columns.to_list()

['대학코드', '제목', '학과', '과정', '년도', 'BERT_SCORE', 'RNN_SCORE', 'href']

In [9]:
def make_abstract_review_df(UNIV_CODE):
    file_path = f"./data_sentiment/abstract/{UNIV_CODE}_review_abstract_sentiment.csv"
    df_abstract = pd.read_csv(file_path, encoding="utf-8")
    df_abstract_review = df_abstract[["제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
    df_abstract_review["대학코드"] = UNIV_CODE
    _abstract_review = df_abstract[["제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
    df_abstract_review["대학코드"] = UNIV_CODE
    df_abstract_review = df_abstract_review[["대학코드","제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
    return df_abstract_review

In [10]:
df1 = make_abstract_review_df("DK000003")
df1.head()

,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,href
0,DK000003,1년의 휘게!,건축학과,학부,2018-2019,1,0.414879,/partner/expReport.asp?id=16597&page=1&bgbn=R
1,DK000003,1년의 Hygge,국제학과,학부,2019,1,0.523097,/partner/expReport.asp?id=16079&page=1&bgbn=R
2,DK000003,코펜하겐 대학교에서의 한 학기! 최고!,경제학과,학부,2019,1,0.931257,/partner/expReport.asp?id=16028&page=1&bgbn=R
3,DK000003,인생에서 가장 행복했던 1년!,심리학과,학부,2019,1,0.967065,/partner/expReport.asp?id=15938&page=1&bgbn=R
4,DK000003,코펜하겐대학교 교환 가세요ㅠㅠㅠㅠ 고민하지 마세요!,언론홍보영상학부,학부,2019,0,0.651389,/partner/expReport.asp?id=15829&page=1&bgbn=R


In [11]:
df2 = make_abstract_review_df("CN000016")
df2.head()

,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,href
0,CN000016,"홍콩대,",UIC,학부,2018,0,0.523097,/partner/expReport.asp?id=16495&page=1&bgbn=R
1,CN000016,홍콩대 교환학생,언더우드국제학부,학부,2019,0,0.523097,/partner/expReport.asp?id=16326&page=1&bgbn=R
2,CN000016,홍콩대학교에서의 한 학기,응용통계학과,학부,2019,1,0.744596,/partner/expReport.asp?id=15804&page=1&bgbn=R
3,CN000016,홍콩대학교 교환 후기,언더우드학과,학부,2019,0,0.640901,/partner/expReport.asp?id=15751&page=1&bgbn=R
4,CN000016,홍콩대학교에서의 한학기,불어불문학과,학부,2019,1,0.744596,/partner/expReport.asp?id=15710&page=1&bgbn=R


In [12]:
df_concat = pd.concat([df1,df2])
df_concat.sample(5)

,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,href
60,DK000003,university of copenhagen - 전옥희,의공학부,학부,2006-2007,1,0.631811,/partner/expReport.asp?id=2616&page=7&bgbn=R
10,CN000016,"나의 리틀 포레스트, 홍콩에서의 교환생활",아시아/정치외교,학부,2018,0,0.725982,/partner/expReport.asp?id=14731&page=2&bgbn=R
30,DK000003,고생한만큼 즐거웠던 코펜하겐!,신문방송학과,학부,2011,1,0.920942,/partner/expReport.asp?id=6650&page=4&bgbn=R
29,CN000016,A World-Class University in Asia's World City,Underwood International College,학부,2014-2015,1,0.524887,/partner/expReport.asp?id=12238&page=3&bgbn=R
38,DK000003,"코펜하겐, 언제가는 꼭 돌아가고 싶은 그곳",언더우드국제대학,학부,2010-2011,1,0.894196,/partner/expReport.asp?id=5203&page=4&bgbn=R


In [13]:
# listing up dataframes to concat
list_df = []
for csv_file in abstract_yonsei_reviews:
    # extract file name
    file_name = csv_file.split("/")[-1]
    file_name_without_ext = file_name.split(".")[0]

    # extract university code
    univ_code = file_name_without_ext.split("_")[0]

    df_temp = make_abstract_review_df(univ_code)
    list_df.append(df_temp)

print(len(list_df))
list_df[:3]

470


[       대학코드                제목               학과  과정    년도  BERT_SCORE  \
 0  730       리투아니아에서 행복했던 4개월  사회복지학과/언론홍보영상학부  학부  2019           1   
 
    RNN_SCORE                                           href  
 0   0.752021  /partner/expReport.asp?id=16663&page=1&bgbn=R  ,
        대학코드                    제목    학과  과정      년도  BERT_SCORE  RNN_SCORE  \
 0  AR000004  아르헨티나 교환학생(어학연수프로그램)  건축학과  학부  2013-1           0    0.39967   
 
                                            href  
 0  /partner/expReport.asp?id=6947&page=1&bgbn=R  ,
         대학코드                                        제목        학과  과정  \
 0   AT000001                       쿠프슈타인에서 노후 50년 땡겨쓰기      경영학과  학부   
 1   AT000001                           평화로운 쿠프에서의 한 학기        경제  학부   
 2   AT000001                   소중한 경험, 잊지 못할 기억, 쿠프슈타인   언론홍보영상학  학부   
 3   AT000001                        언제나 마음속의 고향인 쿠프슈타인    중어중문학과  학부   
 4   AT000001                    자연이 아름다운 쿠프슈타인에서의 한 학기    정치외교학과  학부   
 5   AT000001           

In [14]:
# concat dataframe
df_abstract_wrangle = pd.concat(list_df)
print(len(df_abstract_wrangle.index))
df_abstract_wrangle.head(10)

11183


,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,href
0,730,리투아니아에서 행복했던 4개월,사회복지학과/언론홍보영상학부,학부,2019,1,0.752021,/partner/expReport.asp?id=16663&page=1&bgbn=R
0,AR000004,아르헨티나 교환학생(어학연수프로그램),건축학과,학부,2013-1,0,0.399670,/partner/expReport.asp?id=6947&page=1&bgbn=R
0,AT000001,쿠프슈타인에서 노후 50년 땡겨쓰기,경영학과,학부,2017-2018,0,0.416185,/partner/expReport.asp?id=15098&page=1&bgbn=R
1,AT000001,평화로운 쿠프에서의 한 학기,경제,학부,2017ㅡ2018,1,0.816743,/partner/expReport.asp?id=14709&page=1&bgbn=R
2,AT000001,"소중한 경험, 잊지 못할 기억, 쿠프슈타인",언론홍보영상학,학부,2017,1,0.854657,/partner/expReport.asp?id=14264&page=1&bgbn=R
3,AT000001,언제나 마음속의 고향인 쿠프슈타인,중어중문학과,학부,2017,1,0.834800,/partner/expReport.asp?id=14181&page=1&bgbn=R
4,AT000001,자연이 아름다운 쿠프슈타인에서의 한 학기,정치외교학과,학부,2017,1,0.870668,/partner/expReport.asp?id=14102&page=1&bgbn=R
5,AT000001,여유로웠던 쿠프슈타인에서의 한 학기,행정학과,학부,2017,1,0.816663,/partner/expReport.asp?id=14094&page=1&bgbn=R
6,AT000001,쿠프슈타인에서의 한 학기,경제학부,학부,2016,1,0.731432,/partner/expReport.asp?id=13866&page=1&bgbn=R
7,AT000001,여유롭고 소중했던 쿠프슈타인에서의 시간,경제학부,학부,2016,1,0.852968,/partner/expReport.asp?id=13822&page=1&bgbn=R


In [15]:
# check for duplicate rows
df_abstract_wrangle = df_abstract_wrangle.drop_duplicates()
print(len(df_abstract_wrangle.index))

11183


In [16]:
# export wrangled df abstract concat dataframe
df_abstract_wrangle.to_csv("./data_wrangled/df_abstract_wrangle.csv",encoding="utf-8", index=False)

## Wrangle University Dataframe

In [17]:
def yield_BERT_SUM(UNIV_CODE):
    file_path = f"./data_sentiment/abstract/{UNIV_CODE}_review_abstract_sentiment.csv"
    dataframe_input = pd.read_csv(file_path, encoding="utf-8")
    bert_sum = dataframe_input["BERT_SCORE"].sum()
    return bert_sum

In [18]:
def yield_RNN_SUM(UNIV_CODE):
    file_path = f"./data_sentiment/abstract/{UNIV_CODE}_review_abstract_sentiment.csv"
    dataframe_input = pd.read_csv(file_path, encoding="utf-8")
    rnn_sum = dataframe_input["RNN_SCORE"].sum()
    return rnn_sum

In [19]:
def yield_no_of_students(UNIV_CODE):
    dataframe_input = make_abstract_review_df(UNIV_CODE)
    no_of_students = len(dataframe_input.index)
    return no_of_students

In [20]:
# University of Copenhagen Satisfaction Level
print(yield_BERT_SUM("DK000003"))
print(yield_RNN_SUM("DK000003"))

65
49.14911192655563


In [21]:
# University of Hongkong Satisfaction Level
print(yield_BERT_SUM("CN000016"))
print(yield_RNN_SUM("CN000016"))

41
43.059615552425385


In [22]:
# read university dataframe
df_univ = pd.read_csv("./data/univ_db_full.csv", encoding="utf-8")
df_univ.sample(5)

,level_0,index,No,대학명,국가,파견구분,협정형태,href,visitation
483,483,47,48,CSU Fresno,UNITED STATES,교환학생,교환대학,/partner/expReport.asp?ucode=US000026&bgbn=A,Exists
220,220,4,5,Luiss Guido Carli,ITALY,교환학생,교환대학,/partner/expReport.asp?ucode=IT000008&bgbn=A,Exists
494,494,58,59,CUNY - Baruch College,UNITED STATES,교환학생,교환대학,/partner/expReport.asp?ucode=US000046&bgbn=A,Exists
143,143,19,20,Universite de Franche-Comte,FRANCE,교환학생,ISEP,/partner/expReport.asp?ucode=FR000014&bgbn=A,Exists
474,474,38,39,Columbus State University,UNITED STATES,교환학생,ISEP,/partner/expReport.asp?ucode=US000054&bgbn=A,None


In [23]:
# make column for university code
series_query = df_univ["href"].str.split("=", expand=True)[1]
series_univ_code = series_query.str.split("&", expand=True)[0]
df_univ["대학코드"] = series_univ_code
df_univ.head()

,level_0,index,No,대학명,국가,파견구분,협정형태,href,visitation,대학코드
0,0,0,1,Kabul University,AFGHANISTAN,교환학생,교환대학,/partner/expReport.asp?ucode=AF000001&bgbn=A,None,AF000001
1,1,0,1,Universidad Blas Pascal,ARGENTINA,교환학생,ISEP,/partner/expReport.asp?ucode=AR000001&bgbn=A,None,AR000001
2,2,1,2,Universidad Catolica de Cordoba,ARGENTINA,교환학생,ISEP,/partner/expReport.asp?ucode=AR000002&bgbn=A,None,AR000002
3,3,2,3,Universidad de Palermo,ARGENTINA,교환학생,ISEP,/partner/expReport.asp?ucode=AR000003&bgbn=A,None,AR000003
4,4,3,4,Universidad del Salvador,ARGENTINA,교환학생,ISEP,/partner/expReport.asp?ucode=AR000004&bgbn=A,Exists,AR000004


In [24]:
# reorder columns, delete unnecessary columns
df = df_univ[["대학코드", "국가", "대학명", "파견구분", "협정형태", "visitation", "href"]]
df.head(10)

,대학코드,국가,대학명,파견구분,협정형태,visitation,href
0,AF000001,AFGHANISTAN,Kabul University,교환학생,교환대학,None,/partner/expReport.asp?ucode=AF000001&bgbn=A
1,AR000001,ARGENTINA,Universidad Blas Pascal,교환학생,ISEP,None,/partner/expReport.asp?ucode=AR000001&bgbn=A
2,AR000002,ARGENTINA,Universidad Catolica de Cordoba,교환학생,ISEP,None,/partner/expReport.asp?ucode=AR000002&bgbn=A
3,AR000003,ARGENTINA,Universidad de Palermo,교환학생,ISEP,None,/partner/expReport.asp?ucode=AR000003&bgbn=A
4,AR000004,ARGENTINA,Universidad del Salvador,교환학생,ISEP,Exists,/partner/expReport.asp?ucode=AR000004&bgbn=A
5,AR000005,ARGENTINA,University of Buenos Aires,교환학생,교환대학,None,/partner/expReport.asp?ucode=AR000005&bgbn=A
6,AU000019,AUSTRALIA,Australian National University,교환학생,교환대학,Exists,/partner/expReport.asp?ucode=AU000019&bgbn=A
7,AU000001,AUSTRALIA,Bond University,교환학생,교환대학,Exists,/partner/expReport.asp?ucode=AU000001&bgbn=A
8,AU000002,AUSTRALIA,Curtin University,교환학생,교환대학ISEP,Exists,/partner/expReport.asp?ucode=AU000002&bgbn=A
9,AU000003,AUSTRALIA,Curtin University of Technology,교환학생,NaN,Exists,/partner/expReport.asp?ucode=AU000003&bgbn=A


In [25]:
# sort out unvisited universities
df_ever_been = df.loc[df['visitation'] == "Exists"]
df_ever_been.head()

,대학코드,국가,대학명,파견구분,협정형태,visitation,href
4,AR000004,ARGENTINA,Universidad del Salvador,교환학생,ISEP,Exists,/partner/expReport.asp?ucode=AR000004&bgbn=A
6,AU000019,AUSTRALIA,Australian National University,교환학생,교환대학,Exists,/partner/expReport.asp?ucode=AU000019&bgbn=A
7,AU000001,AUSTRALIA,Bond University,교환학생,교환대학,Exists,/partner/expReport.asp?ucode=AU000001&bgbn=A
8,AU000002,AUSTRALIA,Curtin University,교환학생,교환대학ISEP,Exists,/partner/expReport.asp?ucode=AU000002&bgbn=A
9,AU000003,AUSTRALIA,Curtin University of Technology,교환학생,NaN,Exists,/partner/expReport.asp?ucode=AU000003&bgbn=A


In [26]:
# number of visited universities in Yonsei history
df_ever_been.shape

(470, 7)

In [27]:
# wrangle university dataframe
def make_univ_df():
    # read university dataset csv file
    df_univ = pd.read_csv("./data/univ_db_full.csv", encoding="utf-8")

    # get university code
    series_query = df_univ["href"].str.split("=", expand=True)[1]
    series_univ_code = series_query.str.split("&", expand=True)[0]
    df_univ["대학코드"] = series_univ_code
    
    # Making column values for BERT_SUM, RNN_SUM, STUDENT_NO
    list_bert_sum = []
    list_rnn_sum = []
    list_student_no = []
    
    # iterrating over rows of university datframe
    for index, row in df_univ.iterrows():

        # if university is not ever visited, input NaN values
        if row["visitation"] == "None":
            list_bert_sum.append(np.NaN)
            list_rnn_sum.append(np.NaN)
            list_student_no.append(np.NaN)

        # if university is visited, input BERT_SCORE, RNN_SCORE's sum and input number of visitations
        if row["visitation"] == "Exists":
            list_bert_sum.append(yield_BERT_SUM(row["대학코드"]))
            list_rnn_sum.append(yield_RNN_SUM(row["대학코드"]))
            list_student_no.append(yield_no_of_students(row["대학코드"]))
    
    # Make Columns
    df_univ["BERT_SUM"] = list_bert_sum
    df_univ["RNN_SUM"] = list_rnn_sum
    df_univ["NUM_VISITS"] = list_student_no
    
    # Select and order columns
    df = df_univ[["대학코드", "국가", "대학명", "파견구분", "협정형태", "NUM_VISITS", "BERT_SUM", "RNN_SUM", "href"]]
    return df

In [28]:
df_univ_wrangled = make_univ_df()

In [29]:
print(df_univ_wrangled.shape)
df_univ_wrangled.head(10)

(745, 9)


,대학코드,국가,대학명,파견구분,협정형태,NUM_VISITS,BERT_SUM,RNN_SUM,href
0,AF000001,AFGHANISTAN,Kabul University,교환학생,교환대학,NaN,NaN,NaN,/partner/expReport.asp?ucode=AF000001&bgbn=A
1,AR000001,ARGENTINA,Universidad Blas Pascal,교환학생,ISEP,NaN,NaN,NaN,/partner/expReport.asp?ucode=AR000001&bgbn=A
2,AR000002,ARGENTINA,Universidad Catolica de Cordoba,교환학생,ISEP,NaN,NaN,NaN,/partner/expReport.asp?ucode=AR000002&bgbn=A
3,AR000003,ARGENTINA,Universidad de Palermo,교환학생,ISEP,NaN,NaN,NaN,/partner/expReport.asp?ucode=AR000003&bgbn=A
4,AR000004,ARGENTINA,Universidad del Salvador,교환학생,ISEP,1.0,0.0,0.399670,/partner/expReport.asp?ucode=AR000004&bgbn=A
5,AR000005,ARGENTINA,University of Buenos Aires,교환학생,교환대학,NaN,NaN,NaN,/partner/expReport.asp?ucode=AR000005&bgbn=A
6,AU000019,AUSTRALIA,Australian National University,교환학생,교환대학,27.0,17.0,17.952732,/partner/expReport.asp?ucode=AU000019&bgbn=A
7,AU000001,AUSTRALIA,Bond University,교환학생,교환대학,19.0,16.0,13.359809,/partner/expReport.asp?ucode=AU000001&bgbn=A
8,AU000002,AUSTRALIA,Curtin University,교환학생,교환대학ISEP,7.0,5.0,4.657787,/partner/expReport.asp?ucode=AU000002&bgbn=A
9,AU000003,AUSTRALIA,Curtin University of Technology,교환학생,NaN,16.0,14.0,9.696670,/partner/expReport.asp?ucode=AU000003&bgbn=A


In [30]:
df_univ_wrangled.columns

Index(['대학코드', '국가', '대학명', '파견구분', '협정형태', 'NUM_VISITS', 'BERT_SUM',
       'RNN_SUM', 'href'],
      dtype='object')

In [31]:
df_univ_wrangled.shape

(745, 9)

In [32]:
# check for duplicate rows
df_univ_wrangled = df_univ_wrangled.drop_duplicates()
df_univ_wrangled.shape

(745, 9)

In [33]:
# export wrangled df abstract concat dataframe
df_univ_wrangled.to_csv("./data_wrangled/df_univ_wrangled.csv",encoding="utf-8", index=False)

## Sort universities according to BERT and RNN average score

In [34]:
# GET BERT and RNN average
df_univ_wrangled["BERT_AVG"] = df_univ_wrangled["BERT_SUM"] / df_univ_wrangled["NUM_VISITS"]
df_univ_wrangled["RNN_AVG"] = df_univ_wrangled["RNN_SUM"] / df_univ_wrangled["NUM_VISITS"]

In [35]:
df_univ_wrangled
df_univ_wrangled.sort_values(["BERT_AVG"], ascending=False)

,대학코드,국가,대학명,파견구분,협정형태,NUM_VISITS,BERT_SUM,RNN_SUM,href,BERT_AVG,RNN_AVG
545,US000102,UNITED STATES,Marshall University,교환학생,ISEP,1.0,1.0,0.663917,/partner/expReport.asp?ucode=US000102&bgbn=A,1.0,0.663917
401,GB000003,UNITED KINGDOM,Napier University,교환학생,ISEP,1.0,1.0,0.846949,/partner/expReport.asp?ucode=GB000003&bgbn=A,1.0,0.846949
171,DE000007,GERMANY,Leuphana University of Lunerburg,교환학생,USAC,1.0,1.0,0.773299,/partner/expReport.asp?ucode=DE000007&bgbn=A,1.0,0.773299
509,US000068,UNITED STATES,Fort Hays State University,교환학생,ISEP,1.0,1.0,0.513952,/partner/expReport.asp?ucode=US000068&bgbn=A,1.0,0.513952
400,GB000002,UNITED KINGDOM,Middlesex University,방문학생SAP(정규),SAF,1.0,1.0,0.731432,/partner/expReport.asp?ucode=GB000002&bgbn=A,1.0,0.731432
...,...,...,...,...,...,...,...,...,...,...,...
740,VN000001,VIETNAM,National Economics University of Hanoi,교환학생,NaN,NaN,NaN,NaN,/partner/expReport.asp?ucode=VN000001&bgbn=A,NaN,NaN
741,VN000003,VIETNAM,"Vietnam National University, Hanoi",교환학생,교환대학,NaN,NaN,NaN,/partner/expReport.asp?ucode=VN000003&bgbn=A,NaN,NaN
742,VN000005,VIETNAM,"Vietnam National University, Ho Chi Minh City",NaN,교환대학,NaN,NaN,NaN,/partner/expReport.asp?ucode=VN000005&bgbn=A,NaN,NaN
743,VN000006,VIETNAM,Vin University,NaN,교환대학,NaN,NaN,NaN,/partner/expReport.asp?ucode=VN000006&bgbn=A,NaN,NaN


In [61]:
# Looking at cases with enough number of visits
NUMBER_OF_VISITS = 5
df_enough_visits = df_univ_wrangled.loc[df_univ_wrangled['NUM_VISITS'] >= NUMBER_OF_VISITS]
print(df_enough_visits.shape)
df_enough_visits.head()

(308, 11)


,대학코드,국가,대학명,파견구분,협정형태,NUM_VISITS,BERT_SUM,RNN_SUM,href,BERT_AVG,RNN_AVG
6,AU000019,AUSTRALIA,Australian National University,교환학생,교환대학,27.0,17.0,17.952732,/partner/expReport.asp?ucode=AU000019&bgbn=A,0.629630,0.664916
7,AU000001,AUSTRALIA,Bond University,교환학생,교환대학,19.0,16.0,13.359809,/partner/expReport.asp?ucode=AU000001&bgbn=A,0.842105,0.703148
8,AU000002,AUSTRALIA,Curtin University,교환학생,교환대학ISEP,7.0,5.0,4.657787,/partner/expReport.asp?ucode=AU000002&bgbn=A,0.714286,0.665398
9,AU000003,AUSTRALIA,Curtin University of Technology,교환학생,NaN,16.0,14.0,9.696670,/partner/expReport.asp?ucode=AU000003&bgbn=A,0.875000,0.606042
10,AU000004,AUSTRALIA,Deakin University,교환학생,교환대학ISEPUSAC,24.0,18.0,17.425120,/partner/expReport.asp?ucode=AU000004&bgbn=A,0.750000,0.726047


In [65]:
df_enough_visits_sort = df_enough_visits.sort_values(by=["BERT_AVG"], ascending=False)
df_enough_visits_sort.reset_index()

,index,대학코드,국가,대학명,파견구분,협정형태,NUM_VISITS,BERT_SUM,RNN_SUM,href,BERT_AVG,RNN_AVG
0,415,GB000010,UNITED KINGDOM,University of Edinburgh,교환학생방문학생SAP(정규),교환대학SAF,7.0,7.0,5.607286,/partner/expReport.asp?ucode=GB000010&bgbn=A,1.000000,0.801041
1,475,US000055,UNITED STATES,Cornell College,교환학생,교환대학,10.0,10.0,5.366041,/partner/expReport.asp?ucode=US000055&bgbn=A,1.000000,0.536604
2,221,IT000003,ITALY,Universita Cattolica del Sacro Cuore,교환학생,ISEP,6.0,6.0,3.092777,/partner/expReport.asp?ucode=IT000003&bgbn=A,1.000000,0.515463
3,246,JP000017,JAPAN,Nagasaki University,교환학생,교환대학,7.0,7.0,4.619841,/partner/expReport.asp?ucode=JP000017&bgbn=A,1.000000,0.659977
4,284,MX000004,MEXICO,Instituto Tecnologico y de Estudios Superiores...,교환학생,교환대학ISEP,7.0,7.0,5.665700,/partner/expReport.asp?ucode=MX000004&bgbn=A,1.000000,0.809386
...,...,...,...,...,...,...,...,...,...,...,...,...
303,172,DE000008,GERMANY,Ludwig-Maximilians-Universitat Munchen,교환학생,교환대학,31.0,13.0,21.730792,/partner/expReport.asp?ucode=DE000008&bgbn=A,0.419355,0.700993
304,408,GB000039,UNITED KINGDOM,University of Birmingham,NaN,교환대학ISEP,5.0,2.0,2.948399,/partner/expReport.asp?ucode=GB000039&bgbn=A,0.400000,0.589680
305,82,CN000007,CHINA,Nanjing University,교환학생,교환대학,10.0,4.0,6.720541,/partner/expReport.asp?ucode=CN000007&bgbn=A,0.400000,0.672054
306,383,TW000005,TAIWAN,"National Taiwan University, College of Management",교환학생,교환대학,6.0,2.0,3.883967,/partner/expReport.asp?ucode=TW000005&bgbn=A,0.333333,0.647328


## Sort according to BERT and RNN average score combined

In [67]:
# GET BERT and RNN average
df_univ_wrangled["BERT_PLUS_RNN_SCORE"] = df_univ_wrangled["BERT_AVG"] + df_univ_wrangled["RNN_AVG"]
df_univ_wrangled.sort_values(["BERT_PLUS_RNN_SCORE"], ascending=False)

,대학코드,국가,대학명,파견구분,협정형태,NUM_VISITS,BERT_SUM,RNN_SUM,href,BERT_AVG,RNN_AVG,BERT_PLUS_RNN_SCORE
506,US000065,UNITED STATES,Elmhurst College,교환학생,ISEP,1.0,1.0,0.988630,/partner/expReport.asp?ucode=US000065&bgbn=A,1.0,0.988630,1.988630
54,CA000008,CANADA,Thompson Rivers University,교환학생,ISEP,1.0,1.0,0.950352,/partner/expReport.asp?ucode=CA000008&bgbn=A,1.0,0.950352,1.950352
117,FI000005,FINLAND,The University of Turku,교환학생,ISEP,1.0,1.0,0.948164,/partner/expReport.asp?ucode=FI000005&bgbn=A,1.0,0.948164,1.948164
578,US000133,UNITED STATES,Northwestern State University of Louisiana,교환학생,ISEP,1.0,1.0,0.945189,/partner/expReport.asp?ucode=US000133&bgbn=A,1.0,0.945189,1.945189
493,US000033,UNITED STATES,CSU Stanislaus,교환학생,교환대학,1.0,1.0,0.943452,/partner/expReport.asp?ucode=US000033&bgbn=A,1.0,0.943452,1.943452
...,...,...,...,...,...,...,...,...,...,...,...,...
740,VN000001,VIETNAM,National Economics University of Hanoi,교환학생,NaN,NaN,NaN,NaN,/partner/expReport.asp?ucode=VN000001&bgbn=A,NaN,NaN,NaN
741,VN000003,VIETNAM,"Vietnam National University, Hanoi",교환학생,교환대학,NaN,NaN,NaN,/partner/expReport.asp?ucode=VN000003&bgbn=A,NaN,NaN,NaN
742,VN000005,VIETNAM,"Vietnam National University, Ho Chi Minh City",NaN,교환대학,NaN,NaN,NaN,/partner/expReport.asp?ucode=VN000005&bgbn=A,NaN,NaN,NaN
743,VN000006,VIETNAM,Vin University,NaN,교환대학,NaN,NaN,NaN,/partner/expReport.asp?ucode=VN000006&bgbn=A,NaN,NaN,NaN


In [68]:
# Looking at cases with enough number of visits
NUMBER_OF_VISITS = 5
df_enough_visits = df_univ_wrangled.loc[df_univ_wrangled['NUM_VISITS'] >= NUMBER_OF_VISITS]
print(df_enough_visits.shape)
df_enough_visits.head()

(308, 12)


,대학코드,국가,대학명,파견구분,협정형태,NUM_VISITS,BERT_SUM,RNN_SUM,href,BERT_AVG,RNN_AVG,BERT_PLUS_RNN_SCORE
6,AU000019,AUSTRALIA,Australian National University,교환학생,교환대학,27.0,17.0,17.952732,/partner/expReport.asp?ucode=AU000019&bgbn=A,0.629630,0.664916,1.294546
7,AU000001,AUSTRALIA,Bond University,교환학생,교환대학,19.0,16.0,13.359809,/partner/expReport.asp?ucode=AU000001&bgbn=A,0.842105,0.703148,1.545253
8,AU000002,AUSTRALIA,Curtin University,교환학생,교환대학ISEP,7.0,5.0,4.657787,/partner/expReport.asp?ucode=AU000002&bgbn=A,0.714286,0.665398,1.379684
9,AU000003,AUSTRALIA,Curtin University of Technology,교환학생,NaN,16.0,14.0,9.696670,/partner/expReport.asp?ucode=AU000003&bgbn=A,0.875000,0.606042,1.481042
10,AU000004,AUSTRALIA,Deakin University,교환학생,교환대학ISEPUSAC,24.0,18.0,17.425120,/partner/expReport.asp?ucode=AU000004&bgbn=A,0.750000,0.726047,1.476047


In [71]:
df_enough_visits_sort = df_enough_visits.sort_values(by=["BERT_PLUS_RNN_SCORE"], ascending=False)
df_ranking = df_enough_visits_sort.reset_index()
df_ranking.head(15)

,index,대학코드,국가,대학명,파견구분,협정형태,NUM_VISITS,BERT_SUM,RNN_SUM,href,BERT_AVG,RNN_AVG,BERT_PLUS_RNN_SCORE
0,376,CH000004,SWITZERLAND,University of Lausanne,교환학생,교환대학ISEP,8.0,8.0,6.659102,/partner/expReport.asp?ucode=CH000004&bgbn=A,1.000000,0.832388,1.832388
1,284,MX000004,MEXICO,Instituto Tecnologico y de Estudios Superiores...,교환학생,교환대학ISEP,7.0,7.0,5.665700,/partner/expReport.asp?ucode=MX000004&bgbn=A,1.000000,0.809386,1.809386
2,415,GB000010,UNITED KINGDOM,University of Edinburgh,교환학생방문학생SAP(정규),교환대학SAF,7.0,7.0,5.607286,/partner/expReport.asp?ucode=GB000010&bgbn=A,1.000000,0.801041,1.801041
3,361,SE000003,SWEDEN,Kristianstad University,교환학생,교환대학,49.0,47.0,39.341540,/partner/expReport.asp?ucode=SE000003&bgbn=A,0.959184,0.802889,1.762072
4,264,JP000033,JAPAN,University of Tokyo,교환학생,교환대학,10.0,9.0,8.218766,/partner/expReport.asp?ucode=JP000033&bgbn=A,0.900000,0.821877,1.721877
5,547,US000104,UNITED STATES,Maryville College,교환학생,교환대학,28.0,27.0,21.112275,/partner/expReport.asp?ucode=US000104&bgbn=A,0.964286,0.754010,1.718296
6,521,US000079,UNITED STATES,Hobart & William Smith College,교환학생,교환대학,28.0,26.0,21.590119,/partner/expReport.asp?ucode=US000079&bgbn=A,0.928571,0.771076,1.699647
7,27,AT000001,AUSTRIA,FH Kufstein Tirol University of Applied Sciences,교환학생,교환대학,40.0,36.0,31.848505,/partner/expReport.asp?ucode=AT000001&bgbn=A,0.900000,0.796213,1.696213
8,731,US000273,UNITED STATES,Westminster College,교환학생,교환대학,23.0,21.0,17.890283,/partner/expReport.asp?ucode=US000273&bgbn=A,0.913043,0.777838,1.690882
9,87,CN000010,CHINA,Shandong University,교환학생,교환대학,8.0,7.0,6.468337,/partner/expReport.asp?ucode=CN000010&bgbn=A,0.875000,0.808542,1.683542


In [72]:
df_ranking.tail(15)

,index,대학코드,국가,대학명,파견구분,협정형태,NUM_VISITS,BERT_SUM,RNN_SUM,href,BERT_AVG,RNN_AVG,BERT_PLUS_RNN_SCORE
293,220,IT000008,ITALY,Luiss Guido Carli,교환학생,교환대학,20.0,10.0,13.335463,/partner/expReport.asp?ucode=IT000008&bgbn=A,0.500000,0.666773,1.166773
294,628,US000179,UNITED STATES,Thunderbird School of Global Management,교환학생,교환대학,8.0,5.0,4.247788,/partner/expReport.asp?ucode=US000179&bgbn=A,0.625000,0.530974,1.155974
295,334,RU000002,RUSSIAN FEDERATION,Moscow State University,교환학생,교환대학,29.0,14.0,19.249699,/partner/expReport.asp?ucode=RU000002&bgbn=A,0.482759,0.663783,1.146541
296,404,GB000033,UNITED KINGDOM,"SOAS, University of London",교환학생,교환대학,9.0,4.0,6.251262,/partner/expReport.asp?ucode=GB000033&bgbn=A,0.444444,0.694585,1.139029
297,486,US000021,UNITED STATES,CSU Los Angeles,교환학생,교환대학,8.0,4.0,5.026329,/partner/expReport.asp?ucode=US000021&bgbn=A,0.500000,0.628291,1.128291
298,126,FR000004,FRANCE,Ecole Superieure des Sciences Economiques et C...,교환학생,교환대학,7.0,3.0,4.874636,/partner/expReport.asp?ucode=FR000004&bgbn=A,0.428571,0.696377,1.124948
299,172,DE000008,GERMANY,Ludwig-Maximilians-Universitat Munchen,교환학생,교환대학,31.0,13.0,21.730792,/partner/expReport.asp?ucode=DE000008&bgbn=A,0.419355,0.700993,1.120348
300,187,CN000004,HONG KONG,Hong Kong Polytechnic University,교환학생,교환대학,53.0,23.0,36.354918,/partner/expReport.asp?ucode=CN000004&bgbn=A,0.433962,0.685942,1.119904
301,257,JP000026,JAPAN,Sophia University,교환학생,교환대학,7.0,3.0,4.519462,/partner/expReport.asp?ucode=JP000026&bgbn=A,0.428571,0.645637,1.074209
302,82,CN000007,CHINA,Nanjing University,교환학생,교환대학,10.0,4.0,6.720541,/partner/expReport.asp?ucode=CN000007&bgbn=A,0.400000,0.672054,1.072054


In [73]:
df_ranking.to_csv("./data_wrangled/df_ranking.csv",encoding="utf-8", index=False)